In [3]:
!pip install groq langchain openai streamlit gradio pandas nltk PyMuPDF langchain-community reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 91.8 MB/s eta 0:00:00


In [4]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [12]:
import os
import fitz
import nltk
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

nltk.download("punkt")

groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatOpenAI(
    openai_api_key=groq_api_key,
    model_name="mixtral-8x7b-32768",
    temperature=0.7,
    base_url="https://api.groq.com/openai/v1"
)


def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a given PDF file.
    """
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

def analyze_resume(resume_text, job_description):
    """
    Sends the resume text and job description to the LLM and retrieves an analysis.
    """
    prompt = f"""
    Given the following resume:
    {resume_text}

    And the following job description:
    {job_description}

    Extract the following:
    - Candidate's Name
    - Key Skills
    - Years of Experience
    - Education
    - A suitability score (1-10) based on job requirements.

    Format the response in a structured JSON format.
    """

    response = llm([HumanMessage(content=prompt)])
    return response.content

def process_resumes(resume_folder, job_description):
    """
    Loops through all resumes in a folder, extracts text, and runs analysis.
    """
    results = []
    for file_name in os.listdir(resume_folder):
        if file_name.endswith(".pdf"):
            resume_path = os.path.join(resume_folder, file_name)
            resume_text = extract_text_from_pdf(resume_path)
            analysis = analyze_resume(resume_text, job_description)
            results.append({"Candidate": file_name, "Analysis": analysis})

    return pd.DataFrame(results)

job_description = "We are looking for a software engineer with Python, NLP, and machine learning experience."

resume_folder = "./resumes"
results_df = process_resumes(resume_folder, job_description)
print(results_df)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


         Candidate                                           Analysis
0  data_resume.pdf  {\n  "candidate_name": "Nayab Irfan",\n  "key_...


In [14]:
# to get results in an excel sheet
import pandas as pd
print("Resume Screening Results")
print(results_df)
from IPython.display import display
display(results_df)

results_df.to_csv('resume_screening_results.csv', index=False)

Resume Screening Results
         Candidate                                           Analysis
0  data_resume.pdf  {\n  "candidate_name": "Nayab Irfan",\n  "key_...


,Candidate,Analysis
0,data_resume.pdf,"{\n ""candidate_name"": ""Nayab Irfan"",\n ""key_..."


In [27]:
import os
import re
import fitz
import nltk
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

nltk.download("punkt")

# Set API key for Groq
os.environ["GROQ_API_KEY"] = "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"

# Initialize the LLM model with Groq API
llm = ChatOpenAI(
    openai_api_key=os.getenv("GROQ_API_KEY"),
    model_name="mixtral-8x7b-32768",
    temperature=0.7,
    base_url="https://api.groq.com/openai/v1"
)

def extract_text_from_pdf(pdf_path):
    """Extracts text from a given PDF file."""
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)
    return text.strip()

def extract_candidate_name(resume_text):
    """Attempts to extract the candidate's name using regex (assuming common resume formats)."""
    name_patterns = [
        r"Name:\s*([A-Za-z\s]+)",
        r"([A-Z][a-z]+ [A-Z][a-z]+)",
        r"([A-Z][a-z]+\s[A-Z]\.\s[A-Z][a-z]+)",
    ]

    for pattern in name_patterns:
        match = re.search(pattern, resume_text)
        if match:
            return match.group(1).strip()

    return "Unknown Candidate"

    title_style = ParagraphStyle(
        'TitleStyle', parent=styles['Title'], fontSize=18, textColor=colors.darkblue, spaceAfter=10
    )
    heading_style = ParagraphStyle(
        'HeadingStyle', parent=styles['Heading2'], fontSize=14, textColor=colors.black, spaceAfter=6, bold=True
    )
    normal_style = ParagraphStyle(
        'NormalStyle', parent=styles['Normal'], fontSize=12, textColor=colors.darkgray
    )

def analyze_resume(resume_text):
    """Analyzes the resume and provides structured insights using LLM."""
    prompt = f"""
    Analyze the following resume and extract structured details:

    {resume_text}

    Output format (JSON):
    {{
      "Candidate_Name": "<Extracted Name>",
      "Key_Skills": ["Skill1", "Skill2", "Skill3"],
      "Years_of_Experience": <Number>,
      "Education": ["Degree1", "Degree2"],
      "Suitable_Roles": ["Role1", "Role2", "Role3"],
      "Suitability_Score": <Score out of 10>,
      "Career_Advice": "<Detailed career advice based on skills and experience>"
    }}
    """

    response = llm([HumanMessage(content=prompt)]).content

    try:
        structured_data = eval(response)
    except:
        print("Error parsing LLM response. Check output format.")
        structured_data = {
            "Candidate_Name": extract_candidate_name(resume_text),
            "Key_Skills": [],
            "Years_of_Experience": "N/A",
            "Education": [],
            "Suitable_Roles": [],
            "Suitability_Score": "N/A",
            "Career_Advice": "Analysis could not be completed."
        }

    return structured_data

def generate_pdf_report(resume_folder):
    """Generates a detailed AI-powered resume screening report in PDF format."""
    doc = SimpleDocTemplate("analysis_report.pdf", pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    for file_name in os.listdir(resume_folder):
        if file_name.endswith(".pdf"):
            resume_path = os.path.join(resume_folder, file_name)
            resume_text = extract_text_from_pdf(resume_path)
            analysis = analyze_resume(resume_text)

            name = analysis.get("Candidate_Name", "Unknown Candidate")
            skills = ", ".join(analysis.get("Key_Skills", []))
            experience = analysis.get("Years_of_Experience", "N/A")
            education = ", ".join(analysis.get("Education", []))
            roles = ", ".join(analysis.get("Suitable_Roles", []))
            score = analysis.get("Suitability_Score", "N/A")
            advice = analysis.get("Career_Advice", "No advice available.")

            story.append(Paragraph(f"Resume Analysis for {name}", styles['Title']))
            story.append(Spacer(1, 12))
            story.append(Paragraph(f"Candidate Name: {name}", styles['Normal']))
            story.append(Paragraph(f"Key Skills: {skills}", styles['Normal']))
            story.append(Paragraph(f"Years of Experience: {experience}", styles['Normal']))
            story.append(Paragraph(f"Education: {education}", styles['Normal']))
            story.append(Paragraph(f"Suitable Roles: {roles}", styles['Normal']))
            story.append(Paragraph(f"Suitability Score: {score}/10", styles['Normal']))
            story.append(Spacer(1, 12))
            story.append(Paragraph(f"Personalized Career Advice:", styles['h2']))
            story.append(Paragraph(advice, styles['Normal']))
            story.append(Spacer(1, 24))  # Extra space before next resume

    doc.build(story)
    print("AI-Powered Resume Screening Report saved as analysis_report.pdf")

resume_folder = "./resumes"
generate_pdf_report(resume_folder)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AI-Powered Resume Screening Report saved as analysis_report.pdf


In [29]:
import os
import re
import fitz
import nltk
import pandas as pd
import streamlit as st
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors

# Download NLTK tokenizer
nltk.download("punkt")

# Set API key for Groq
os.environ["GROQ_API_KEY"] = "your_groq_api_key_here"

# Initialize LLM
llm = ChatOpenAI(
    openai_api_key=os.getenv("GROQ_API_KEY"),
    model_name="mixtral-8x7b-32768",
    temperature=0.7,
    base_url="https://api.groq.com/openai/v1"
)

# Streamlit UI
st.set_page_config(page_title="AI Resume Analyzer", layout="centered")
st.title("AI Resume Analyzer")

# Upload PDF File
uploaded_file = st.file_uploader("Upload a Resume (PDF)", type=["pdf"])

def extract_text_from_pdf(pdf_path):
    """Extract text from a given PDF file."""
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)
    return text.strip()

def analyze_resume(resume_text):
    """Analyze resume content using LLM."""
    prompt = f"""
    Analyze the following resume and extract structured details:

    {resume_text}

    Output format (JSON):
    {{
      "Candidate_Name": "<Extracted Name>",
      "Key_Skills": ["Skill1", "Skill2", "Skill3"],
      "Years_of_Experience": <Number>,
      "Education": ["Degree1", "Degree2"],
      "Suitable_Roles": ["Role1", "Role2", "Role3"],
      "Suitability_Score": <Score out of 10>,
      "Career_Advice": "<Detailed career advice based on skills and experience>"
    }}
    """

    response = llm([HumanMessage(content=prompt)]).content
    try:
        structured_data = eval(response)
    except:
        structured_data = {
            "Candidate_Name": "Unknown",
            "Key_Skills": [],
            "Years_of_Experience": "N/A",
            "Education": [],
            "Suitable_Roles": [],
            "Suitability_Score": "N/A",
            "Career_Advice": "Analysis failed."
        }
    return structured_data

def generate_pdf_report(analysis):
    """Generate a detailed resume analysis report in PDF."""
    pdf_path = "resume_analysis_report.pdf"
    doc = SimpleDocTemplate(pdf_path, pagesize=letter)
    styles = getSampleStyleSheet()

    title_style = ParagraphStyle('TitleStyle', parent=styles['Title'], fontSize=18, textColor=colors.darkblue)
    heading_style = ParagraphStyle('HeadingStyle', parent=styles['Heading2'], fontSize=14, textColor=colors.black, bold=True)
    normal_style = ParagraphStyle('NormalStyle', parent=styles['Normal'], fontSize=12, textColor=colors.darkgray)

    story = []
    name = analysis.get("Candidate_Name", "Unknown Candidate")
    skills = ", ".join(analysis.get("Key_Skills", []))
    experience = analysis.get("Years_of_Experience", "N/A")
    education = ", ".join(analysis.get("Education", []))
    roles = ", ".join(analysis.get("Suitable_Roles", []))
    score = analysis.get("Suitability_Score", "N/A")
    advice = analysis.get("Career_Advice", "No advice available.")

    story.append(Paragraph(f"Resume Analysis Report", title_style))
    story.append(Spacer(1, 12))
    story.append(Paragraph(f"Candidate Name: {name}", heading_style))
    story.append(Paragraph(f"Key Skills: {skills}", normal_style))
    story.append(Paragraph(f"Years of Experience: {experience}", normal_style))
    story.append(Paragraph(f"Education: {education}", normal_style))
    story.append(Paragraph(f"Suitable Roles: {roles}", normal_style))
    story.append(Paragraph(f"Suitability Score: {score}/10", normal_style))
    story.append(Spacer(1, 12))
    story.append(Paragraph(f"Career Advice:", heading_style))
    story.append(Paragraph(advice, normal_style))
    story.append(Spacer(1, 24))

    doc.build(story)
    return pdf_path

if uploaded_file:
    with open("uploaded_resume.pdf", "wb") as f:
        f.write(uploaded_file.getbuffer())

    st.success("Resume Uploaded Successfully!")

    resume_text = extract_text_from_pdf("uploaded_resume.pdf")

    # Analyze Resume
    with st.spinner("Analyzing Resume..."):
        analysis = analyze_resume(resume_text)

    # Show Results
    st.subheader("Resume Analysis Result")
    st.write(f"**Candidate Name:** {analysis['Candidate_Name']}")
    st.write(f"**Key Skills:** {', '.join(analysis['Key_Skills'])}")
    st.write(f"**Years of Experience:** {analysis['Years_of_Experience']}")
    st.write(f"**Education:** {', '.join(analysis['Education'])}")
    st.write(f"**Suitable Roles:** {', '.join(analysis['Suitable_Roles'])}")
    st.write(f"**Suitability Score:** {analysis['Suitability_Score']}/10")
    st.write(f"**Career Advice:** {analysis['Career_Advice']}")

    # Generate PDF Report
    with st.spinner("📄 Generating PDF Report..."):
        pdf_path = generate_pdf_report(analysis)

    # Provide Download Button
    with open(pdf_path, "rb") as pdf_file:
        st.download_button(
            label="Download Report",
            data=pdf_file,
            file_name="Resume_Analysis_Report.pdf",
            mime="application/pdf"
        )


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-02-15 16:02:06.903 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 16:02:06.905 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 16:02:06.968 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-15 16:02:06.969 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 16:02:06.972 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 16:02:06.974 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mod

In [31]:
!streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.247.121.113:8501

  Stopping...
  Stopping...


In [36]:
import os
import fitz
import gradio as gr
import json
import nltk
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

nltk.download("punkt")

# ✅ Faster Model + Lower Temperature
llm = ChatOpenAI(
    openai_api_key=os.getenv("GROQ_API_KEY"),
    model_name="gpt-3.5-turbo",  # Faster than Mixtral-8x7b
    temperature=0.7,
)

def extract_text_from_pdf(pdf_path):
    """Extracts key sections from the resume."""
    doc = fitz.open(pdf_path)
    all_text = "\n".join(page.get_text("text") for page in doc)

    sections = all_text.split("\n")
    keywords = ["Experience", "Skills", "Education", "Summary", "Projects"]
    filtered_text = "\n".join([s for s in sections if any(k in s for k in keywords)])

    return filtered_text if filtered_text else all_text[:1000]

def analyze_resume(resume_text):
    """Uses LLM to analyze and extract structured details."""
    prompt = f"""
    Summarize the resume below and extract key details in JSON:

    {resume_text[:1500]}

    JSON Format:
    {{
      "Candidate_Name": "<Extracted Name>",
      "Key_Skills": ["Skill1", "Skill2"],
      "Years_of_Experience": <Number>,
      "Education": ["Degree1", "Degree2"],
      "Suitable_Roles": ["Role1", "Role2"],
      "Suitability_Score": <Score out of 10>,
      "Career_Advice": "<Short Career Advice>"
    }}
    """

    response = llm([HumanMessage(content=prompt)]).content
    try:
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "AI response parsing failed. Try again!"}

def generate_pdf_report(resume_text):
    """Creates a well-structured PDF with AI insights."""
    analysis = analyze_resume(resume_text)
    if "error" in analysis:
        return analysis["error"]

    pdf_filename = "resume_analysis.pdf"
    doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    story.append(Paragraph("Resume Analysis Report", styles['Title']))
    story.append(Spacer(1, 12))

    for key, value in analysis.items():
        if isinstance(value, list):
            value = ", ".join(value)
        story.append(Paragraph(f"<b>{key.replace('_', ' ')}:</b> {value}", styles['Normal']))
        story.append(Spacer(1, 10))

    doc.build(story)
    return pdf_filename

def process_resume(pdf_file):
    """Handles file upload, extracts text, and generates the analysis report."""
    resume_text = extract_text_from_pdf(pdf_file.name)
    pdf_filename = generate_pdf_report(resume_text)
    return f"Analysis complete! Download your report:", pdf_filename

# Optimize Gradio Launch
iface = gr.Interface(
    fn=process_resume,
    inputs=gr.File(label="Upload Resume (PDF)"),
    outputs=[gr.Text(), gr.File(label="Download Report")],
    title="⚡ AI-Powered Resume Analyzer",
    description="Upload a resume and get a structured AI-generated analysis instantly!"
)

iface.launch(share=True)  # Faster execution


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5711a666116f66ca36.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
